# NB6 Comparison Between Different Classifiers

## Objectives

Compare with other classification methods
   * Decision trees with **`tree.DecisionTreeClassifier()`**
   * K-nearest neighbours with **`neighbors.KNeighborsClassifier()`**
   * Random forests with **`ensemble.RandomForestClassifier()`**
   * Perceptron (both gradient and stochastic gradient) with **`mlxtend.classifier.Perceptron`**
   * Multilayer perceptron network (both gradient and stochastic gradient) with **`mlxtend.classifier.MultiLayerPerceptron`**

## Inputs

* outputs/datasets/cleaned/data.csv

## Outputs

* accuracy_baseline.png, final_model_accuracy.png, final_model_classification.png, model_training_accuracy_knn.png,
* model_training_accuracy_svc.png, scaled_algorithm.png, score_uncertainty.png, six_classifiers.png, standardize_data.png
* compare_algorithm.jpeg, scaled_algorithm.jpeg


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os

current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Automate the ML process using pipelines 

There are standard workflows in a machine learning project that can be automated. In Python scikit-learn, Pipelines help to clearly define and automate these workflows.
* Pipelines help overcome common problems like data leakage in your test harness. 
* Python scikit-learn provides a Pipeline utility to help automate machine learning workflows.
* Pipelines work by allowing for a linear sequence of data transforms to be chained together culminating in a modelling process that can be evaluated.

### Data Preparation and Modelling Pipeline

In [ ]:
import matplotlib.pyplot as plt

# Create a pipeline that standardizes the data then creates a model
# Load libraries for data processing
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from scipy.stats import norm

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# visualization
import seaborn as sns

plt.style.use("fivethirtyeight")
sns.set_style("white")

plt.rcParams["figure.figsize"] = (8, 4)

### Evaluate Some Algorithms
Now it is time to create some models of the data and estimate their accuracy on unseen data. Here is what we are going to cover in this step:
1. Separate out a validation dataset.
2. Setup the test harness to use 10-fold cross validation.
3. Build 5 different models  
4. Select the best model

## 1.0 Validation Dataset

In [ ]:
# Load data
data = pd.read_csv("outputs/datasets/cleaned/data.csv", index_col=False)
data.drop("Unnamed: 0", axis=1, inplace=True)

# Split-out validation dataset
array = data.values
X = array[:, 1:31]
y = array[:, 0]

# Divide records in training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Transform the class labels from their original string representation
# (M and B) into integers
le = LabelEncoder()
y = le.fit_transform(y)

## 2.0 Evaluate Algorithms: Baseline

In [ ]:
# Spot-Check Algorithms
models = []
models.append(("LR", LogisticRegression(solver="lbfgs", max_iter=1000)))
models.append(("LDA", LinearDiscriminantAnalysis()))
models.append(("KNN", KNeighborsClassifier()))
models.append(("CART", DecisionTreeClassifier()))
models.append(("NB", GaussianNB()))
models.append(("SVM", SVC()))

# Test options and evaluation metric
num_folds = 10
num_instances = len(X_train)
seed = 7
scoring = "accuracy"

# Test options and evaluation metric
num_folds = 10
num_instances = len(X_train)
seed = 7
scoring = "accuracy"
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    print(
        "\n➔ 10-Fold cross-validation accurcay score for the training data for six classifiers"
    )

#### Observation
> The results suggest That both Logistic Regression and LDA may be worth further study. These are just mean accuracy values. It is always wise to look at the distribution of accuracy values calculated across cross validation folds. We can do that graphically using box and whisker plots.


In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle("Algorithm Comparison")
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.savefig("outputs/nb6/compare_algorithm.jpeg")
plt.show()

#### Observation
> The results show a similar tight distribution for all classifiers except SVM which is encouraging, suggesting low variance. The poor results for SVM are surprising.

> It is possible the varied distribution of the attributes may have an effect on the accuracy of algorithms such as SVM. In the next section we will repeat this spot-check with a standardized copy of the training dataset.


### 2.1 Evaluate Algorithms: Standardize Data

In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(
    ("ScaledLR", Pipeline([("Scaler", StandardScaler()), ("LR", LogisticRegression())]))
)
pipelines.append(
    (
        "ScaledLDA",
        Pipeline([("Scaler", StandardScaler()), ("LDA", LinearDiscriminantAnalysis())]),
    )
)
pipelines.append(
    (
        "ScaledKNN",
        Pipeline([("Scaler", StandardScaler()), ("KNN", KNeighborsClassifier())]),
    )
)
pipelines.append(
    (
        "ScaledCART",
        Pipeline([("Scaler", StandardScaler()), ("CART", DecisionTreeClassifier())]),
    )
)
pipelines.append(
    ("ScaledNB", Pipeline([("Scaler", StandardScaler()), ("NB", GaussianNB())]))
)
pipelines.append(
    ("ScaledSVM", Pipeline([("Scaler", StandardScaler()), ("SVM", SVC())]))
)

results = []
names = []

for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle("Scaled Algorithm Comparison")
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.savefig("outputs/nb6/scaled_algorithm.jpeg")
plt.show()

#### Observations
> The results show that standardization of the data has lifted the skill of SVM to be the most accurate algorithm tested so far.

The results suggest digging deeper into the SVM and LDA and LR algorithms. It is very likely that configuration beyond the default may yield even more accurate models.

## 3.0 Algorithm Tuning
In this section we investigate tuning the parameters for three algorithms that show promise from the spot-checking in the previous section: LR, LDA and SVM.

### Tuning hyper-parameters - SVC estimator

In [ ]:
# Make Support Vector Classifier Pipeline
pipe_svc = Pipeline(
    [
        ("scl", StandardScaler()),
        ("pca", PCA(n_components=2)),
        ("clf", SVC(probability=True, verbose=False)),
    ]
)

# Fit Pipeline to training Data
pipe_svc.fit(X_train, y_train)

# print('--> Fitted Pipeline to training Data')

scores = cross_val_score(
    estimator=pipe_svc, X=X_train, y=y_train, cv=10, n_jobs=1, verbose=0
)
print("--> Model Training Accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

# Tune Hyperparameters
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {"clf__C": param_range, "clf__kernel": ["linear"]},
    {"clf__C": param_range, "clf__gamma": param_range, "clf__kernel": ["rbf"]},
]
gs = GridSearchCV(
    estimator=pipe_svc, param_grid=param_grid, scoring="accuracy", cv=10, n_jobs=1
)
gs = gs.fit(X_train, y_train)
print("➔ Tuned Parameters Best Score: ", gs.best_score_)
print("➔ Best Parameters: \n", gs.best_params_)

### Tuning the hyper-parameters - k-NN hyperparameters
 For your standard k-NN implementation, there are two primary hyperparameters that you’ll want to tune:

* The number of neighbors k.
* The distance metric/similarity function.

Both of these values can dramatically affect the accuracy of your k-NN classifier. Grid object is ready to do 10-fold cross validation on a KNN model using classification accuracy as the evaluation metric
In addition, there is a parameter grid to repeat the 10-fold cross validation process 30 times
Each time, the n_neighbors parameter should be given a different value from the list
We can't give GridSearchCV just a list
We've to specify n_neighbors should take on 1 through 30
You can set n_jobs = -1 to run computations in parallel (if supported by your computer and OS) 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

pipe_knn = Pipeline(
    [
        ("scl", StandardScaler()),
        ("pca", PCA(n_components=2)),
        ("clf", KNeighborsClassifier()),
    ]
)

# Fit Pipeline to training Data
pipe_knn.fit(X_train, y_train)

scores = cross_val_score(estimator=pipe_knn, X=X_train, y=y_train, cv=10, n_jobs=1)
print("--> Model Training Accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

# Tune Hyperparameters
param_range = range(1, 31)
param_grid = [{"clf__n_neighbors": param_range}]
# Instantiate the grid
gs = GridSearchCV(estimator=pipe_knn, param_grid=param_grid, cv=10, scoring="accuracy")
gs = gs.fit(X_train, y_train)
print("➔ Tuned Parameters Best Score: ", gs.best_score_)
print("➔ Best Parameters: \n", gs.best_params_)

### Finalize Model

In [ ]:
# Use best parameters
clf_svc = gs.best_estimator_

# Get Final Scores
clf_svc.fit(X_train, y_train)
scores = cross_val_score(estimator=clf_svc, X=X_train, y=y_train, cv=10, n_jobs=1)
print(
    "--> Final Model Training Accuracy: %.3f +/- %.3f"
    % (np.mean(scores), np.std(scores))
)

print("--> Final Accuracy on Test set: %.5f" % clf_svc.score(X_test, y_test))

In [ ]:
clf_svc.fit(X_train, y_train)
y_pred = clf_svc.predict(X_test)


print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Summary

Worked through a classification predictive modeling machine learning problem from end-to-end using Python. Specifically, the steps covered were:

1. Problem Definition (Breast Cancer data).

2. Loading the Dataset.

3. Analyze Data (same scale but different distributions of data).

    * Evaluate Algorithms (KNN looked good).
    * Evaluate Algorithms with Standardization (KNN and SVM looked good).

4. Algorithm Tuning (K=19 for KNN was good, SVM with an RBF kernel and C=100 was best).. 
5. Finalize Model (use all training data and confirm using validation dataset)
